#### Chris Sutton
#### 
#### Lab#3

In [1]:
# A brief summary of my work:
# This week I used regEx to parse and pull out and order my data in my data strucutres. I created a vocabulary
# from all word in the document via a set. I stored each document in a list then used collections.counter to provide 
# a frequency of occurence. created an inverse document table via a list of dict that contained tuples of (DF, (docID, tf)).
# while my code for parts a-e executed quickly it ran in to significant run time for (f). I ran some cells for 6hrs+. 
# I quickly attempted to integrate stop words into my code but I was too committed from the previous sections, this would 
# have caused a significant rewrite of my code which there wasn't time for. While the code was runing I considered the use
# of some recursive seach/sort methods but this also would have caused significant headache in an already defined code. 

#### (a) Build in-memory inverted files. Inverted files are the primary data structure to support the efficient determination
#### of which documents contain specified terms. The objective of this first phase is to process a corpus, much as in the first
#### lab exercise. However, this time you will not only create a dictionary with document frequency counts for terms, but each
#### dictionary value should also store a postings list (list of docids and term counts) for that term. Document frequency is
#### the number of records in the postings list. If you are judicious about your memory use, you can store all these lists in
#### memory for the collections that we will use. For part (a) use the TIME dataset, which is very small. The documents are
#### in a single file, one document per line. The format is a tab-separated (TSV) file with "docid [tab] text". Some lines may
#### be long.

#### 2 points: Show the posting list tuples (docid, term count) for up to the first 10 (numerically lowest) docids for
#### the terms (a) computer, (b) thailand, and (c) rockets
#### 2 points: Print the DF and IDF values for the same three words, as well as the total number of documents that
#### you processed from the file.
#### 2 points: Report how long it took to process the input file of documents and create the dictionary and postings
#### lists (in minutes and seconds)

In [2]:
import re
import collections
import math
import time

In [3]:
start = time.time()
stringText = open('time-documents.txt','rt',encoding='utf-8').read()
# calulate vocabulary of documents 
# vocabulary set
terms = set(re.findall(r"\w+",stringText, re.MULTILINE))

#list of each document
listOfDocs= re.findall(r"^\d+\t(.*\n)",stringText, re.MULTILINE)
#list of document numbers
listofDocNumbers = [int(i) for i in re.findall(r"^\d+",stringText, re.MULTILINE)]

#count of terms in each document (list of doc term frequecy)
docTermCounts = []
for l in listOfDocs:
    docTermCounts.append(collections.Counter(re.findall(r"\w+",l, re.MULTILINE)))
      
#build table of DF and (doc, term counts)          
table =collections.defaultdict(list)
# calculate DF (first element in table list)
i=0 # DF counter

# # enter edit
# for n, t in enumerate(terms):
#     for number in range(len(docTermCounts)):
#         if t in docTermCounts[number].keys():
#             i+=1
#             table[t]=[i]
#     i=0  
    
# # list documents and term counts (tuples in table list (document number, term count))
# for n, t in enumerate(terms):
#     for l in range(len(docTermCounts)):
#         if t in docTermCounts[l].keys():
#             table[t].append((listofDocNumbers[l],docTermCounts[l][t]))
# print(table["THE"])

# # end edit

# start zip edit
i=0 # DF counter
for t in terms:
    for l in docTermCounts:
        if t in l.keys():
            i+=1
            table[t]=[i]
    i=0  
# list documents and term counts (tuples in table list (document number, term count))
for t in terms:
    for l,m in zip(docTermCounts, listofDocNumbers):
        if t in l.keys():
            table[t].append((m,l[t]))
# end zip edit


end = time.time()

In [4]:
# print ten lowest docIds = 'COMPUTER'
sorted(table['COMPUTER'][1:11],key=lambda x: x[0])

[(308, 1)]

In [5]:
# print ten lowest docIds = 'THAILAND'
sorted(table['THAILAND'][1:11], key=lambda x: x[0]) 
# table['THAILAND']

[(203, 1),
 (243, 5),
 (280, 14),
 (396, 1),
 (449, 1),
 (498, 1),
 (516, 1),
 (534, 5),
 (543, 12),
 (544, 2)]

In [6]:
# print ten lowest docIds = 'ROCKETS'
sorted(table['ROCKETS'][1:11], key=lambda x: x[0])

[(27, 1),
 (117, 1),
 (186, 1),
 (313, 6),
 (404, 1),
 (464, 2),
 (495, 1),
 (509, 2),
 (545, 2)]

In [7]:
def IDF (listDoc, listOfTerm_DF_DocID_TF, termString):
    return math.log2(len(listDoc)/listOfTerm_DF_DocID_TF[termString][0])

In [8]:
# print DF and IDF = 'COMPUTER'
print ('DF :',table['COMPUTER'][0], '  IDF  :' ,IDF(listofDocNumbers, table, 'COMPUTER'))

DF : 1   IDF  : 8.72451385311995


In [9]:
# print DF and IDF = 'THAILAND'
print ('DF :',table['THAILAND'][0], '  IDF  :' ,IDF(listofDocNumbers, table, 'THAILAND'))

DF : 11   IDF  : 5.2650822344826524


In [10]:
# print DF and IDF = 'ROCKETS'
print ('DF :',table['ROCKETS'][0], '  IDF  :' ,IDF(listofDocNumbers, table, 'ROCKETS'))

DF : 9   IDF  : 5.554588851677638


In [11]:
# total number of documents processed
len(listofDocNumbers)

423

In [12]:
# time to import input file of documents and create the dictionary and postings
print( end - start)

3.428626298904419


#### (b) Document vector length. To compute cosine scores you need to apply TF/IDF term weights and compute vector
#### lengths for both documents and queries. We will start with documents. We do not have a list of all terms contained in
#### each document. Instead the inverted file from part (a) has lists of docids for each term. However, there is a simple
#### algorithm to compute document vector lengths for all documents in parallel, by iterating over all postings lists a single
#### time. You then store (say in a separate hashtable) document vector lengths as a single real number. Here is the efficient
#### practical algorithm to compute the document vector length, which is the square-root of the sum of the squares of the
#### weights.
#### 6 points: Print document vector lengths for the first 10 (numerically lowest) docids

In [13]:
def docVectorLength(listDoc, listOfTerm_DF_DocID_TF ):
    doclens = {}
    for term in listOfTerm_DF_DocID_TF:
        for docid, tf in listOfTerm_DF_DocID_TF[term][1:]:
            weight = tf * IDF(listDoc,listOfTerm_DF_DocID_TF,term)
            if docid in doclens.keys():
                doclens[docid] += weight **2
            else:
                doclens[docid] = weight **2
#     print(sorted(doclens.items()))
    for id,sum in doclens.items():
        doclens[id] = math.sqrt(sum)
    return doclens

In [14]:
docVector= docVectorLength(listofDocNumbers, table)
sorted(docVectorLength(listofDocNumbers, table).items(), key=lambda item: item[0])[:10]
# print(docVectorLength(listofDocNumbers, table))


[(17, 142.98567609258976),
 (18, 63.781528783328575),
 (19, 135.90483657626655),
 (20, 71.25308668419089),
 (21, 146.06350621019592),
 (23, 129.1497104390011),
 (24, 219.14554737046032),
 (25, 71.20475021266738),
 (26, 117.39747064506734),
 (27, 73.94031818838077)]

#### (c) Query representation. Read in the file of queries (also a tab-separated file "queryid [tab] text"). Tokenize the text
#### the same way you tokenized the document collection. Keep term counts (i.e., the query term frequency) for each indexing
#### term you find. Note, some terms may not be found in the dictionary -- if so, then just ignore them. Compute a query
#### vector length similar to what was done in (b), but this will be easier since you have the entire query vector easily at hand.
#### 2 points: Print TF/IDF weights with each query term for the first query (only)
#### 2 points: Print the query vector length for the first query (only)

In [15]:
def quertyTf (filePath):
    queryStringOfText = open(filePath,'rt',encoding='utf-8').read()
    
    #list of each query
    listOfQuery= re.findall(r"^\d+\t(.*\n)",queryStringOfText, re.MULTILINE)
    listofQueryNumbers = [int(i) for i in re.findall(r"^\d+",queryStringOfText, re.MULTILINE)]
    
    #count terms in each query (list of query term frequecy Dict['word']:value)
    queryTermCounts = []
    for l in listOfQuery:
        queryTermCounts.append(collections.Counter(re.findall(r"\w+",l, re.MULTILINE)))
    return queryTermCounts

In [16]:
listOfWeightsPerQuery=[]
queryVectorLength=[]

# list of tuple of (word, term frequency)
for line in quertyTf('time-queries.txt'):
    sqofVectorLengthcomponents=[]
    dotproduct=[]
    for word in line.items():
    #     if query word not in collection vocabulary, skip
        if word[0] not in terms: #key = word from query in vocabulary
            pass
        else:
    # tf*IDF dot product store in list 
            dotproduct.append(word[1]*IDF(listofDocNumbers, table, word[0]))
    # store vector length components squared 
            sqofVectorLengthcomponents.append((word[1]*IDF(listofDocNumbers, table, word[0]))**2)
#     print('qtf*docIDF =',dotproduct)
# caputre dot product components for a line of query
    listOfWeightsPerQuery.append(dotproduct)
# caputre query vector length per line of query
    queryVectorLength.append(math.sqrt(sum(sqofVectorLengthcomponents)))
    
print('query vector length of first query =',queryVectorLength[0]) 
print('\nqtf*docIDF =',listOfWeightsPerQuery[0])
    

query vector length of first query = 11.847005038012957

qtf*docIDF = [3.169925001442312, 4.4765863396763645, 3.7245138531199498, 0.13955135239879357, 4.402585758232587, 4.402585758232587, 4.200951897062937, 0.003414664412764716, 3.6801197337614964, 0.0, 5.139551352398794]


#### (d) Score documents. Documents in the collection can be scored in parallel by processing each query term one at a time.
#### Walk down the posting list for each query term and add a partial score to an accumulator for each seen docid. The partial
#### score is increased like this:
#### score{docid} += querytf * idf(term) * doctf * idf(term)
#### After processing all query terms you have a sum for many docids. Now divide score{docid} by doclens{docid} and the
#### query vector length. The resulting score is the cosine value and it should lie between 0 and 1.
#### 2 points: Report the time it takes to score all queries (minutes/seconds)
#### 8 points: Cosine scores appear correct


In [17]:
start2 =time.time()
listOfQueryScore=[]
for k in range(len(quertyTf('time-queries.txt'))):
    score={} 
    for word in quertyTf('time-queries.txt')[k].items():
#     if query word not in collection vocabulary, skip
        if word[0] not in terms:
            pass
        else:
            # tf*IDF dot product store in score dict
            for docID, doctf in table[word[0]][1:]:
                if docID in score.keys():
                    score[docID]+=word[1]*(IDF(listofDocNumbers, table, word[0])**2)*doctf/(docVector[docID]*queryVectorLength[k])
                else:
                    score[docID]=word[1]*(IDF(listofDocNumbers, table, word[0])**2)*doctf/(docVector[docID]*queryVectorLength[k])
    listOfQueryScore.append(score)
end2 = time.time()

In [18]:
print(end2-start2)

0.28008270263671875


#### (e) Ranked List. Sort the scores for docids and build a ranked list of the top 50 docids per query. Remember that a
#### higher cosine score is usually a better document. Create an output file named time-YOURJHED.txt that precisely follows
#### the following format:
#### queryid Q0 docid rank score jhed
#### where the six fields are separated by a single space. Here is a description of each of the six required fields:
#### queryid: the number in the first field of the input TSV file
#### Q0: string literal that is always the same (reads like capital queue zero)
#### docid: the numeric id for each document
#### rank: starting at 1 (and not more than 50)
#### score: cosine score (should be between 0 and 1). No scientific notation allowed. Use 4 to 6 digits of precision.
#### jhed: your jhedid (same on each line)
####  4 points: Following formatting directions precisely
####  4 points: Submit ranked list

In [19]:
eSortedList=[]
outputList=[]
# sorted(x.items(), key=lambda item: item[1])
file1 = open("time-CSUTTO16.txt", "w") 
for x in listOfQueryScore:
    eSortedList.append(sorted(x.items(), key=lambda item: item[1], reverse=True)[:50])
count=0
for q in eSortedList:
    count+=1
    rank=0
    for doc,scor in q:
        rank+=1
#         print(doc ,scor, count, rank)
#         outputList.append((count,'Q0',doc,rank,scor,'csutto16'))
        file1.write(str(count)+' '+'Q0'+' '+str(doc)+' '+str(rank)+' '+str(scor)+' '+'csutto16'+'\n')
file1.close()


#### (f) Efficiency. See if your code scales by working on a larger dataset. If you are successful, provide output file fire10-
#### jhedid.txt. The FIRE10 data from the class site is a collection of news stories. The data are available in the zipfile. The
#### corpus is copyrighted1 and cannot be used for commercial purposes. Note that this collection is larger and the content is
#### based around the Indian subcontinent. There are also some oddities - for example, I believe that apostrophe and quote
#### characters were replaced with spaces or deleted. There are approximately 120,000 documents (around 300 MB). Note
#### that there are missing docids in the collection and the queries file does not start at 1. If you have a problem with this
#### larger dataset, don't fret about it too much. Just index as much of it as you can, and describe what you are able to do.
#### 2 points: Report time to build (minutes/seconds) as in (a)
#### 2 points: Report time to score all queries (minutes/seconds) as in (d)
#### 2 points: Provide a reasonable ranked list for these data as in (e)

In [20]:
start3 = time.time()
# import nltk
from nltk.corpus import stopwords
# nltk.download('stopwords')


In [21]:
start = time.time()
stringText = open('fire10-documents.txt','rt',encoding='utf-8').read()
# calulate vocabulary of documents 
# vocabulary set
terms = set(re.findall(r"\w+",stringText, re.MULTILINE))

#list of each document
listOfDocs= re.findall(r"^\d+\t(.*\n)",stringText, re.MULTILINE)
#list of document numbers
listofDocNumbers = [int(i) for i in re.findall(r"^\d+",stringText, re.MULTILINE)]

KeyboardInterrupt: 

In [ ]:
#count of terms in each document (list of doc term frequecy)
docTermCounts = []
for l in listOfDocs:
    docTermCounts.append(collections.Counter(re.findall(r"\w+",l, re.MULTILINE)))
   

In [ ]:
#  6 plus hours run time for this cell until stopped!!!!!  
# I wansn't clear from the lab to use NLTK stop words or to remove anything. The assignment said:
# try this file and report the times. The run time with my previous code are excessive when used
# so I droppped problem (f).
# code below is a mirror image of the code above and is what I intended on using. however, the use of stop words 
# with my current code appears to be a significant rewrite of my previous design


#build table of DF and (doc, term counts)          
table =collections.defaultdict(list)
# calculate DF (first element in table list)
i=0 # DF counter
for t in terms:
    for l in docTermCounts:
        if t in l.keys():
            i+=1
            table[t]=[i]
    i=0  
 

In [ ]:
   
# list documents and term counts (tuples in table list (document number, term count))
for t in terms:
    for l,m in zip(docTermCounts, listofDocNumbers):
        if t in l.keys():
            table[t].append((m,l[t]))

In [ ]:
docVector= docVectorLength(listofDocNumbers, table)

In [ ]:
listOfWeightsPerQuery=[]
queryVectorLength=[]

# list of tuple of (word, term frequency)
for line in quertyTf('fire10-queries.txt'):
    sqofVectorLengthcomponents=[]
    dotproduct=[]
    for word in line.items():
    #     if query word not in collection vocabulary, skip
        if word[0] not in terms: #key = word from query in vocabulary
            pass
        else:
    # tf*IDF dot product store in list 
            dotproduct.append(word[1]*IDF(listofDocNumbers, table, word[0]))
    # store vector length components squared 
            sqofVectorLengthcomponents.append((word[1]*IDF(listofDocNumbers, table, word[0]))**2)
#     print('qtf*docIDF =',dotproduct)
# caputre dot product components for a line of query
    listOfWeightsPerQuery.append(dotproduct)
# caputre query vector length per line of query
    queryVectorLength.append(math.sqrt(sum(sqofVectorLengthcomponents)))
end3 = time.time()

In [ ]:
start4 = time.time()
listOfQueryScore=[]
for k in range(len(quertyTf('fire10-queries.txt'))):
    score={} 
    for word in quertyTf('fire10-queries.txt')[k].items():
#     if query word not in collection vocabulary, skip
        if word[0] not in terms:
            pass
        else:
            # tf*IDF dot product store in score dict
            for docID, doctf in table[word[0]][1:]:
                if docID in score.keys():
                    score[docID]+=word[1]*(IDF(listofDocNumbers, table, word[0])**2)*doctf/(docVector[docID]*queryVectorLength[k])
                else:
                    score[docID]=word[1]*(IDF(listofDocNumbers, table, word[0])**2)*doctf/(docVector[docID]*queryVectorLength[k])
    listOfQueryScore.append(score)
end4=time.time()

In [ ]:
eSortedList=[]
outputList=[]
# sorted(x.items(), key=lambda item: item[1])
# file1 = open("fire-CSUTTO16.txt", "w") 
for x in listOfQueryScore:
    eSortedList.append(sorted(x.items(), key=lambda item: item[1], reverse=True)[:50])
# count=0
# for q in eSortedList:
#     count+=1
#     rank=0
#     for doc,scor in q:
#         rank+=1
# #         print(doc ,scor, count, rank)
# #         outputList.append((count,'Q0',doc,rank,scor,'csutto16'))
#         file1.write(str(count)+' '+'Q0'+' '+str(doc)+' '+str(rank)+' '+str(scor)+' '+'csutto16'+'\n')
# file1.close()

In [ ]:
eSortedList[0]